In [1]:
import os
import numpy as np
import ase
from ase.io.trajectory import Trajectory

In [2]:
from qmlearn.model import QMModel
from qmlearn.drivers.mol import QMMol
from qmlearn.preprocessing import get_train_atoms
from qmlearn.io.hdf5 import DBHDF5

In [3]:
basis = 'cc-pvTZ'
xc = 'lda,vwn_rpa'
method = 'rks'
charge = 0

In [4]:
nsamples=10000
tol = 1E-3
mdtraj = 'h2o_vib.traj'

In [5]:
train_atoms=get_train_atoms(mdtraj, nsamples=nsamples, tol=tol)
nsamples = len(train_atoms)

WARN : Only get 36 samples at 90 step. Maybe you can reduce the 'tol'.


In [6]:
refqmmol = QMMol(atoms = train_atoms[0],  method = method, basis=basis, xc = xc)

In [7]:
prop = ['vext', 'gamma', 'energy', 'forces']
properties= { k: [] for k in prop}

for atoms in train_atoms:
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    gamma=qmmol.engine.gamma
    properties['vext'].append(qmmol.engine.vext)
    properties['gamma'].append(qmmol.engine.gamma)
    properties['energy'].append(qmmol.engine.etotal)
    properties['forces'].append(qmmol.engine.forces)

In [8]:
dbfile = os.path.splitext(mdtraj)[0]+'_QML_set.hdf5'
db = DBHDF5(dbfile, qmmol=refqmmol)

In [9]:
db.write_qmmol(refqmmol)
db.write_images(train_atoms, prefix='train')
db.write_properties(properties, prefix='train')
print(db.names)
db.close()

['rks', 'rks/qmmol', 'rks/train_atoms_36', 'rks/train_props_36']


In [10]:
db = DBHDF5(dbfile, qmmol=refqmmol)
data = db.read_properties(db.get_names('*/train_props_*')[0])
db.close()
data['energy'][0]

-76.09382403593867